# Aranet Data Cleaning and Analysis
The purpose of this notebook is to analysis the basic structure of our loaded aranet4 data to check or missing or duplicate records that could cause discontinuities in our time-series forecasting. We will correct and fill/impute missing values accordingly, and write functions to detect and correct for similar errors for data collected in the future.

## Load Data

In [154]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as datetime

# Load the data
# Load the data from the CSV file
aranet4 = pd.read_csv('../datasets/aranet4.csv')
aranetExp = pd.read_csv('../datasets/aranetExp.csv')

aranetExp['date'] = pd.to_datetime(aranetExp['date'])
aranetExp = aranetExp.rename(columns={'date': 'Datetime'})
aranetExp = aranetExp.drop(columns=['id','time'])
aranetExp = aranetExp.set_index('Datetime')

# Convert the date column to datetime 
aranet4['Datetime'] = pd.to_datetime(aranet4['Datetime'])
aranet4 = aranet4.set_index('Datetime')

In [155]:
aranet4.info(), aranetExp.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 129596 entries, 2024-01-18 17:08:30 to 2024-04-17 18:03:12
Data columns (total 4 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   Carbon dioxide(ppm)        129596 non-null  int64  
 1   Temperature(°F)            129596 non-null  float64
 2   Relative humidity(%)       129472 non-null  float64
 3   Atmospheric pressure(hPa)  127003 non-null  float64
dtypes: float64(3), int64(1)
memory usage: 4.9 MB
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 263 entries, 2024-03-13 13:04:41 to 2024-04-17 13:10:42
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   door1          263 non-null    object
 1   door2          263 non-null    object
 2   hvac           263 non-null    object
 3   subject_count  263 non-null    int64 
dtypes: int64(1), object(3)
memory usage: 10.3+ KB


(None, None)

## Clean Data

We see we have some `null` values in humidity and pressure, we won't need those values for our current experiment, so we can subset for after `2024-03-13 00:00:00`.

In [156]:
# Filter the DataFrame to include all records from "2024-03-14 00:00:00" onwards
aranet4_sub = aranet4.loc["2024-03-14 00:00:00":].copy()
aranet4_sub.info()


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 50044 entries, 2024-03-14 00:00:49 to 2024-04-17 18:03:12
Data columns (total 4 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Carbon dioxide(ppm)        50044 non-null  int64  
 1   Temperature(°F)            50044 non-null  float64
 2   Relative humidity(%)       49973 non-null  float64
 3   Atmospheric pressure(hPa)  49973 non-null  float64
dtypes: float64(3), int64(1)
memory usage: 1.9 MB


In order to join the two data sets we have to put them on a unique common datetime minute index. First, let's round everything to the nearest minute and looking duplicate minutes.

In [157]:
def custom_round_to_nearest_minute(dt_index):
    rounded_index = dt_index.floor('min') + pd.to_timedelta((dt_index.second >= 30).astype(int), unit='min')
    rounded_index.name = dt_index.name  # Preserve the name of the index
    return rounded_index

aranet4_sub.index = custom_round_to_nearest_minute(aranet4_sub.index)
aranetExp.index = custom_round_to_nearest_minute(aranetExp.index)
aranet4_sub.info(), aranetExp.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 50044 entries, 2024-03-14 00:01:00 to 2024-04-17 18:03:00
Data columns (total 4 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Carbon dioxide(ppm)        50044 non-null  int64  
 1   Temperature(°F)            50044 non-null  float64
 2   Relative humidity(%)       49973 non-null  float64
 3   Atmospheric pressure(hPa)  49973 non-null  float64
dtypes: float64(3), int64(1)
memory usage: 1.9 MB
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 263 entries, 2024-03-13 13:05:00 to 2024-04-17 13:11:00
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   door1          263 non-null    object
 1   door2          263 non-null    object
 2   hvac           263 non-null    object
 3   subject_count  263 non-null    int64 
dtypes: int64(1), object(3)
memory usage: 10.3+ KB


(None, None)

Check for duplicates and gaps in the index

In [158]:
import pandas as pd

# Find the indices of the rows where the index is duplicated and the previous indices
duplicate_indices = aranet4_sub.index.duplicated(keep='first')
aranet4_sub[duplicate_indices].index

DatetimeIndex(['2024-03-15 08:21:00', '2024-03-17 02:02:00',
               '2024-03-20 08:38:00', '2024-03-20 16:04:00',
               '2024-03-22 08:24:00', '2024-03-24 08:39:00',
               '2024-03-25 20:19:00', '2024-03-27 08:54:00',
               '2024-03-28 17:56:00', '2024-03-31 06:21:00',
               '2024-03-31 21:57:00', '2024-04-01 01:52:00',
               '2024-04-03 04:28:00', '2024-04-07 00:21:00',
               '2024-04-09 22:20:00', '2024-04-11 16:26:00',
               '2024-04-14 06:34:00', '2024-04-16 07:24:00'],
              dtype='datetime64[ns]', name='Datetime', freq=None)

In [159]:
aranet4_sub.loc['2024-03-15 08:20:00':'2024-03-15 08:22:00']

,Carbon dioxide(ppm),Temperature(°F),Relative humidity(%),Atmospheric pressure(hPa)
Datetime,,,,
2024-03-15 08:20:00,749,69.8,48.0,1015.2
2024-03-15 08:21:00,728,69.9,48.0,1015.1
2024-03-15 08:21:00,760,69.9,48.0,1015.2
2024-03-15 08:22:00,767,70.0,48.0,1015.1


We can downsample the duplicate index records which differ along their values with a `groupby`

In [160]:
aranet4_sub = aranet4_sub.groupby(aranet4_sub.index).mean()

# Find the indices of the rows where the index is duplicated and the previous indices
duplicate_indices = aranet4_sub.index.duplicated(keep='first')
print(aranet4_sub[duplicate_indices].index)
aranet4_sub.loc['2024-03-15 08:20:00':'2024-03-15 08:22:00']

DatetimeIndex([], dtype='datetime64[ns]', name='Datetime', freq=None)


,Carbon dioxide(ppm),Temperature(°F),Relative humidity(%),Atmospheric pressure(hPa)
Datetime,,,,
2024-03-15 08:20:00,749.0,69.8,48.0,1015.20
2024-03-15 08:21:00,744.0,69.9,48.0,1015.15
2024-03-15 08:22:00,767.0,70.0,48.0,1015.10


How about the missing values?

In [161]:
import pandas as pd

def report_missing_values(data, time_interval='1T'):
    """
    Reports the missing values in a time series based on a given time interval.

    Parameters:
    - data: DataFrame or Series with a DateTimeIndex.
    - time_interval: String representing the time interval (default is '1T' for 1 minute).

    Returns:
    - gap_durations: Series containing the durations of the gaps greater than the specified time interval.
    """
    # Calculate the time difference between consecutive records
    time_diff = data.index.to_series().diff()

    # Find the indices where the time difference is greater than the specified time interval
    gaps = time_diff > pd.Timedelta(time_interval)

    # Filter the DataFrame to only include the records immediately after the gaps
    gaps_df = data[gaps]

    # Display the information of the filtered DataFrame
    print("Info about the gaps in the data:")
    gaps_df.info()

    # Extract the time differences for the gaps
    gap_durations = time_diff[gaps]

    # Display the gap durations
    print("\nDurations of the gaps greater than the specified time interval:")
    # print(gap_durations)

    return gap_durations


report_missing_values(aranet4_sub, time_interval='1T')


Info about the gaps in the data:
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 17 entries, 2024-03-17 00:09:00 to 2024-04-15 06:51:00
Data columns (total 4 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Carbon dioxide(ppm)        17 non-null     float64
 1   Temperature(°F)            17 non-null     float64
 2   Relative humidity(%)       16 non-null     float64
 3   Atmospheric pressure(hPa)  16 non-null     float64
dtypes: float64(4)
memory usage: 680.0 bytes

Durations of the gaps greater than the specified time interval:


Datetime
2024-03-17 00:09:00   0 days 00:02:00
2024-03-18 14:41:00   0 days 00:02:00
2024-03-20 10:34:00   0 days 00:02:00
2024-03-21 08:25:00   0 days 00:02:00
2024-03-23 08:33:00   0 days 00:02:00
2024-03-25 18:23:00   0 days 00:02:00
2024-03-26 08:44:00   0 days 00:02:00
2024-03-28 10:30:00   0 days 00:02:00
2024-03-30 05:53:00   0 days 00:02:00
2024-03-31 08:19:00   0 days 00:02:00
2024-03-31 23:55:00   0 days 00:02:00
2024-04-02 02:07:00   0 days 00:02:00
2024-04-03 23:04:00   0 days 00:02:00
2024-04-08 20:07:00   0 days 00:02:00
2024-04-10 22:33:00   0 days 00:02:00
2024-04-12 18:14:00   0 days 00:02:00
2024-04-15 06:51:00   0 days 00:02:00
Name: Datetime, dtype: timedelta64[ns]

We should be able to upsample the missing 1 minute intervals.

Note: **ValueError: cannot reindex on an axis with duplicate labels**

We must address the duplicates we found earlier.

In [162]:
import pandas as pd

# Assuming aranet4_sub is your DataFrame with a DateTimeIndex

# Resample the data to 1-minute intervals
resampled_data = aranet4_sub.resample('1T').asfreq()

# Interpolate the missing values
interpolated_data = resampled_data.interpolate(method='time')

# Now, interpolated_data should have the 2-minute gaps filled.
report_missing_values(interpolated_data, time_interval='1T')

Info about the gaps in the data:
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 0 entries
Freq: T
Data columns (total 4 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Carbon dioxide(ppm)        0 non-null      float64
 1   Temperature(°F)            0 non-null      float64
 2   Relative humidity(%)       0 non-null      float64
 3   Atmospheric pressure(hPa)  0 non-null      float64
dtypes: float64(4)
memory usage: 0.0 bytes

Durations of the gaps greater than the specified time interval:


Series([], Freq: T, Name: Datetime, dtype: timedelta64[ns])

We have taken care of all duplicate and missing indices for the `aranet4_sub`. Next we need to check `aranetExp` for duplicate records because they will be merged onto `aranet4_sub`

In [163]:
# Find the indices of the rows where the index is duplicated and the previous indices
duplicate_indices = aranetExp.index.duplicated(keep=False)
# duplicate_indices = aranetExp.index.duplicated(keep='first')
# aranetExp[duplicate_indices].index
aranetExp[duplicate_indices]

,door1,door2,hvac,subject_count
Datetime,,,,
2024-03-13 13:08:00,closed,open,on,17
2024-03-13 13:08:00,closed,open,off,17
2024-03-18 12:25:00,open,open,off,5
2024-03-18 12:25:00,open,open,off,8
2024-03-19 09:18:00,open,closed,off,15
2024-03-19 09:18:00,open,closed,on,15
2024-03-25 10:18:00,open,open,on,15
2024-03-25 10:18:00,open,open,off,15
2024-04-02 12:34:00,open,open,on,18


We have duplicates on the `DatetimeIndex`, this is expected for this data because of how it is collected. We can't aggregate to downsample so we will shift the last duplicate by 1T time unit. But first we need t

In [164]:
aranetExp_reset = aranetExp.reset_index()

# Drop complete duplicates, keeping only the first occurrence
aranetExp = aranetExp_reset.drop_duplicates(keep='first')

aranetExp = aranetExp.set_index('Datetime')

# Find the indices of the rows where the index is duplicated
duplicate_indices = aranetExp.index.duplicated(keep=False)

# Display rows with duplicate datetime stamps
duplicate_rows = aranetExp[duplicate_indices]
duplicate_rows



,door1,door2,hvac,subject_count
Datetime,,,,
2024-03-13 13:08:00,closed,open,on,17
2024-03-13 13:08:00,closed,open,off,17
2024-03-18 12:25:00,open,open,off,5
2024-03-18 12:25:00,open,open,off,8
2024-03-19 09:18:00,open,closed,off,15
2024-03-19 09:18:00,open,closed,on,15
2024-03-25 10:18:00,open,open,on,15
2024-03-25 10:18:00,open,open,off,15
2024-04-02 12:34:00,open,open,on,18


We see all the duplicates have been dropped an only true duplicate `DatetimeIndex` records remain.

In [165]:
# Find the indices of the rows where the index is duplicated and the previous indices
duplicate_indices = aranetExp.index.duplicated(keep='last')
aranetExp[duplicate_indices]

,door1,door2,hvac,subject_count
Datetime,,,,
2024-03-13 13:08:00,closed,open,on,17
2024-03-18 12:25:00,open,open,off,5
2024-03-19 09:18:00,open,closed,off,15
2024-03-25 10:18:00,open,open,on,15
2024-04-02 12:34:00,open,open,on,18
2024-04-04 12:30:00,open,open,on,15
2024-04-08 12:31:00,open,closed,on,12
2024-04-10 12:31:00,open,open,off,16


We have to use a loop because sometimes when we shift a record forward or backward along the index we encounter another record which must then be moved. Ultimately we want to minimize the level of dispersion. We are sending records backward, so one could say we are adding a partial lag to the series.

In [166]:
loop_count = 0

while True:
    # Find the indices of the rows where the index is duplicated and keep the first occurrence
    duplicate_indices = aranetExp.index.duplicated(keep='first')
    
    # Break the loop if there are no more duplicates
    if not duplicate_indices.any():
        break
    
    # Create a new index by shifting the first occurrence of each duplicate by 1 minute backwards
    new_index = aranetExp.index.values.copy()
    new_index[duplicate_indices] = new_index[duplicate_indices] - pd.Timedelta(minutes=1)
    
    # Update the DataFrame index
    aranetExp.index = pd.DatetimeIndex(new_index)

    # Increment the loop count
    loop_count += 1

# Verify the changes
print("No more duplicates:", not aranetExp.index.duplicated().any())
print("Number of loops used:", loop_count)



No more duplicates: True
Number of loops used: 2


Now that we have removed all duplicates from `aranetExp` we can merge it onto `aranet4_sub`

In [167]:
# Merge aranet4_copy with aranetExp_copy using an exact match on the 'Datetime' index
aranet = pd.merge(aranet4_sub, aranetExp, left_on='Datetime', right_index=True, how='left')


In [168]:
aranet.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 50026 entries, 2024-03-14 00:01:00 to 2024-04-17 18:03:00
Data columns (total 8 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Carbon dioxide(ppm)        50026 non-null  float64
 1   Temperature(°F)            50026 non-null  float64
 2   Relative humidity(%)       49955 non-null  float64
 3   Atmospheric pressure(hPa)  49955 non-null  float64
 4   door1                      247 non-null    object 
 5   door2                      247 non-null    object 
 6   hvac                       247 non-null    object 
 7   subject_count              247 non-null    float64
dtypes: float64(5), object(3)
memory usage: 4.4+ MB


The aranet data is now cleaned and there shouldn't be any gaps or duplicates index records remaining. The data from the lecture experiments can be filled/interpolated within there lecture frames. We can handle that process now. 

In [169]:
import pandas as pd

def label_time_period(df, dayofweek, start_time, end_time, label_name, column_name):
    """
    Labels rows based on specified time intervals and days of the week.

    Parameters:
    - df: DataFrame with a DateTimeIndex.
    - dayofweek: List of integers representing days of the week (Monday=0, Sunday=6) to apply the label.
    - start_time: String in 'HH:MM' format representing the start of the time interval.
    - end_time: String in 'HH:MM' format representing the end of the time interval.
    - label_name: The label to apply for rows within the specified time interval.
    - column_name: The column to store the labels. If the column does not exist, it will be created.

    Returns:
    - None. The function modifies the DataFrame in place.
    """
    def label_row(row):
        if row.name.dayofweek in dayofweek:
            if row.name.time() >= pd.to_datetime(start_time).time() and row.name.time() <= pd.to_datetime(end_time).time():
                return label_name
        return None

    # Ensure the column exists in the DataFrame
    if column_name not in df.columns:
        df[column_name] = None

    # Apply the labeling function
    new_labels = df.apply(label_row, axis=1)
    df[column_name] = df[column_name].combine_first(new_labels)

# Example usage:
# Assuming 'aranet' and 'aranetExp' are your DataFrames with a DateTimeIndex
label_time_period(aranet, [0, 1, 2, 3], '12:15', '13:30', 'Song', 'lecturer')
label_time_period(aranet, [0, 2], '09:00', '10:15', 'Chen', 'lecturer')
label_time_period(aranet, [1, 3], '08:00', '09:15', 'Chen', 'lecturer')


We can write the dataset to a `.csv` at this stage as a simple labeled set.

Let's add some features for future use in our tensors that would other wise result in NaNs. In a way this will allow us to pull data from outside the span of our window into it.

<span style="color:red"> Calculating the moving averages before partitioning the data means that the moving averages for the training set are influenced by the test set data. This is a form of data leakage because when you train the model, it has access to information it wouldn't have in a real-world scenario, where future data points aren't available at the time of prediction.</span>

In [170]:
aranet.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 50026 entries, 2024-03-14 00:01:00 to 2024-04-17 18:03:00
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Carbon dioxide(ppm)        50026 non-null  float64
 1   Temperature(°F)            50026 non-null  float64
 2   Relative humidity(%)       49955 non-null  float64
 3   Atmospheric pressure(hPa)  49955 non-null  float64
 4   door1                      247 non-null    object 
 5   door2                      247 non-null    object 
 6   hvac                       247 non-null    object 
 7   subject_count              247 non-null    float64
 8   lecturer                   3038 non-null   object 
dtypes: float64(5), object(4)
memory usage: 4.8+ MB


In [171]:
aranet.to_csv('../datasets/aranet.csv')

# Rename the columns
aranet.rename(columns={
    'Carbon dioxide(ppm)': 'co2',
    'Temperature(°F)': 'tempF',
    'Relative humidity(%)': 'rhumid',
    'Atmospheric pressure(hPa)': 'atmpr'
}, inplace=True)

def create_rolling_averages(df, columns, intervals):
    for col in columns:
        for interval in intervals:
            new_col_name = f"{col}_ma{interval}"
            df[new_col_name] = df[col].rolling(f"{interval}T").mean()
    return df

# # Define the columns and intervals for the rolling averages
# columns = ['co2', 'tempF', 'rhumid']
# intervals = [5, 10, 15, 30]

# # Apply the function to create the rolling averages
# aranet = create_rolling_averages(aranet, columns, intervals)


# aranet.info()

## Lecture Tensor Set
We can create a dictionary of the lectures and a tensor with a corresponding set of labels for easier access in modeling.

### Lecture Dictionary

In [172]:
aranet_lecturer = aranet[aranet['lecturer'].notnull()].copy()
aranet_lecturer.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3038 entries, 2024-03-14 08:00:00 to 2024-04-17 13:30:00
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   co2            3038 non-null   float64
 1   tempF          3038 non-null   float64
 2   rhumid         3038 non-null   float64
 3   atmpr          3038 non-null   float64
 4   door1          217 non-null    object 
 5   door2          217 non-null    object 
 6   hvac           217 non-null    object 
 7   subject_count  217 non-null    float64
 8   lecturer       3038 non-null   object 
dtypes: float64(5), object(4)
memory usage: 237.3+ KB


In [173]:
aranet_lecturer[:]

,co2,tempF,rhumid,atmpr,door1,door2,hvac,subject_count,lecturer
Datetime,,,,,,,,,
2024-03-14 08:00:00,595.0,65.8,40.0,1017.0,open,open,on,10.0,Chen
2024-03-14 08:01:00,597.0,66.4,39.0,1016.9,NaN,NaN,NaN,NaN,Chen
2024-03-14 08:02:00,584.0,66.8,39.0,1016.9,open,open,off,16.0,Chen
2024-03-14 08:03:00,611.0,67.2,39.0,1017.0,NaN,NaN,NaN,NaN,Chen
2024-03-14 08:04:00,577.0,67.5,39.0,1017.0,NaN,NaN,NaN,NaN,Chen
...,...,...,...,...,...,...,...,...,...
2024-04-17 13:26:00,1019.0,73.1,44.0,1017.4,NaN,NaN,NaN,NaN,Song
2024-04-17 13:27:00,1042.0,73.0,44.0,1017.4,NaN,NaN,NaN,NaN,Song
2024-04-17 13:28:00,1023.0,72.9,44.0,1017.4,NaN,NaN,NaN,NaN,Song


In [174]:
import pandas as pd

# def create_windows(df, window_size, label_column):
#     """
#     Creates fixed-size windows from the DataFrame and labels them based on a specified column.

#     Parameters:
#     - df: DataFrame with a DateTimeIndex.
#     - window_size: Size of the window in minutes.
#     - label_column: Column name to use for labeling the windows.

#     Returns:
#     - windows: Dictionary with labels as keys and lists of windows as values.
#     """
#     windows = {}
#     start = 0

#     while start < len(df):
#         # Calculate the end of the window
#         end = start + window_size

#         # Extract the window
#         window = df.iloc[start:end]

#         # Check if 'door1' column has only NaN values
#         if window['door1'].isna().all():
#             # Move to the next window without adding the current window
#             start = end
#             continue

#         # Get the label for the window
#         label = window[label_column].mode()[0]

#         # Add the window to the dictionary
#         if label not in windows:
#             windows[label] = [window]
#         else:
#             windows[label].append(window)

#         # Move to the next window
#         start = end

#     return windows

# def create_windows(df, window_size, label_column):
#     """
#     Creates fixed-size windows from the DataFrame, ensuring that each window falls within
#     the correct time intervals for the given label and excludes windows where 'door1' is all NaN.

#     Parameters:
#     - df: DataFrame with a DateTimeIndex and pre-labeled rows.
#     - window_size: Size of the window in minutes.
#     - label_column: Column name to use for labeling the windows.

#     Returns:
#     - windows: Dictionary with labels as keys and lists of windows as values.
#     """
#     windows = {}

#     # Iterate over each unique label in the DataFrame
#     unique_labels = df[label_column].dropna().unique()

#     for label in unique_labels:
#         # Filter the DataFrame for the current label
#         label_df = df[df[label_column] == label]
        
#         start = 0
#         while start < len(label_df):
#             end = start + window_size
#             # Ensure not to exceed the DataFrame's length
#             if end > len(label_df):
#                 break

#             window = label_df.iloc[start:end]

#             # Check if 'door1' column has only NaN values
#             if 'door1' in window.columns and window['door1'].isna().all():
#                 start = end  # Skip this window and move to the next
#                 continue

#             # Check if window spans across different days
#             if window.index[0].date() != window.index[-1].date():
#                 start += 1  # Move the window by one minute to exclude the record from the next day
#                 continue

#             # Add the window to the dictionary under the appropriate label
#             if label not in windows:
#                 windows[label] = [window]
#             else:
#                 windows[label].append(window)

#             start = end  # Move to the next window

#     return windows

def create_windows(df, window_size, label_column):
    """
    Creates fixed-size windows from the DataFrame, ensuring that each window falls within
    the correct time intervals for the given label and excludes windows where 'door1' is all NaN.

    Parameters:
    - df: DataFrame with a DateTimeIndex and pre-labeled rows.
    - window_size: Size of the window in minutes.
    - label_column: Column name to use for labeling the windows.

    Returns:
    - windows: Dictionary with labels as keys and lists of windows as values.
    """
    windows = {}

    # Iterate over each unique label in the DataFrame
    unique_labels = df[label_column].dropna().unique()

    for label in unique_labels:
        # Filter the DataFrame for the current label
        label_df = df[df[label_column] == label]
        
        # Group lectures by day
        for day, day_df in label_df.groupby(label_df.index.day):
            start = 0
            while start < len(day_df):
                end = start + window_size
                # Ensure not to exceed the DataFrame's length
                if end > len(day_df):
                    break

                window = day_df.iloc[start:end]

                # Check if 'door1' column has only NaN values
                if 'door1' in window.columns and window['door1'].isna().all():
                    start = end  # Skip this window and move to the next
                    continue

                # Add the window to the dictionary under the appropriate label
                if label not in windows:
                    windows[label] = [window]
                else:
                    windows[label].append(window)

                start = end  # Move to the next window

    return windows

# Example usage
window_size = 76  # 76 minutes
windows = create_windows(aranet_lecturer, window_size, 'lecturer')

# Inspect the windows for a specific label
windows['Song'][1].shape  # First window with the label 'Song'


(76, 9)

In [175]:
total_windows = sum(len(windows_list) for windows_list in windows.values())
print(f"Total number of windows created: {total_windows}")


Total number of windows created: 27


In [176]:
# Initialize a variable to store the shape of the first window for comparison
first_window_shape = None

# Dictionary to store the shapes of windows that do not match the first window's shape
mismatched_shapes = {}

for label, windows_list in windows.items():
    for i, window in enumerate(windows_list):
        if first_window_shape is None:
            first_window_shape = window.shape
        elif window.shape != first_window_shape:
            mismatched_shapes[f"{label} window {i}"] = window.shape

if not mismatched_shapes:
    print(f"All windows have the same shape: {first_window_shape}")
else:
    print("Windows with mismatched shapes:")
    for key, shape in mismatched_shapes.items():
        print(f"{key}: {shape}")


All windows have the same shape: (76, 9)


Now that the windows are created we can forward fill and then backfill `NaN` values. The idea being that the state from one minute carries forward until it reaches a non-Missing record.

In [177]:
for label, windows_list in windows.items():
    for i, window in enumerate(windows_list):
        # Create a copy of the window to avoid modifying a slice in place
        window_copy = window.copy()

        # Forward fill and backfill any remaining missing values
        window_copy.fillna(method='ffill', inplace=True)
        window_copy.fillna(method='bfill', inplace=True)

        # Encode specific values as 1, others as 0
        window_copy['door1'] = (window_copy['door1'] == 'open').astype(int)
        window_copy['door2'] = (window_copy['door2'] == 'open').astype(int)
        window_copy['hvac'] = (window_copy['hvac'] == 'on').astype(int)
        window_copy['lecturer'] = (window_copy['lecturer'] == 'Song').astype(int)

        # Update the window in the dictionary with the modified copy
        windows[label][i] = window_copy


In [178]:
# Check the shapes of the DataFrames after one-hot encoding
for label, windows_list in windows.items():
    print(f"Label: {label}")
    for i, window in enumerate(windows_list):
        print(f"Window {i}: {window.shape}")


Label: Chen
Window 0: (76, 9)
Window 1: (76, 9)
Window 2: (76, 9)
Window 3: (76, 9)
Window 4: (76, 9)
Window 5: (76, 9)
Window 6: (76, 9)
Window 7: (76, 9)
Window 8: (76, 9)
Window 9: (76, 9)
Window 10: (76, 9)
Window 11: (76, 9)
Window 12: (76, 9)
Label: Song
Window 0: (76, 9)
Window 1: (76, 9)
Window 2: (76, 9)
Window 3: (76, 9)
Window 4: (76, 9)
Window 5: (76, 9)
Window 6: (76, 9)
Window 7: (76, 9)
Window 8: (76, 9)
Window 9: (76, 9)
Window 10: (76, 9)
Window 11: (76, 9)
Window 12: (76, 9)
Window 13: (76, 9)


In [179]:
windows['Song'][13].index.minute

Int64Index([15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31,
            32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48,
            49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59,  0,  1,  2,  3,  4,  5,
             6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22,
            23, 24, 25, 26, 27, 28, 29, 30],
           dtype='int64', name='Datetime')

In [180]:
for label, windows_list in windows.items():
    print(f"Label: {label}, Number of windows: {len(windows_list)}")


Label: Chen, Number of windows: 13
Label: Song, Number of windows: 14


In [181]:
def check_windows(windows, lecturer_column):
    """
    Checks each DataFrame within the windows dictionary for specified properties.
    - Ensures each DataFrame has only a single unique lecturer.
    - Verifies that the DateTimeIndex is continuous without unintended breaks strictly greater than 1 minute.

    Parameters:
    - windows: Dictionary with labels as keys and lists of DataFrames as values.
    - lecturer_column: The name of the column to check for unique values.

    Returns:
    - None. Prints the status for each DataFrame.
    """
    all_good = True
    for label, df_list in windows.items():
        for index, df in enumerate(df_list):
            # Check for unique values in the lecturer column
            if df[lecturer_column].nunique() != 1:
                all_good = False
                print(f"Issue found: Label '{label}' at index {index} has multiple lecturer values.")

            # Get the minute component of the DataFrame's index as a pandas Series
            minutes = pd.Series(df.index.minute)
            # Calculate time differences
            time_diffs = minutes.diff().fillna(0)

            # Check for any time differences strictly greater than one minute
            if (time_diffs > 1).any():
                all_good = False
                # Find the actual timestamps where these breaks occur
                breaks = df.index[time_diffs > 1]
                for break_time in breaks:
                    print(f"Issue found: Label '{label}' at index {index} has a significant break at {break_time}.")

    if all_good:
        print("All windows are verified: single unique lecturer and continuous index. Everything is fine.")

# Example usage (after windows have been created):
check_windows(windows, 'lecturer')



All windows are verified: single unique lecturer and continuous index. Everything is fine.


In [188]:
windows['Chen'][12]

,co2,tempF,rhumid,atmpr,door1,door2,hvac,subject_count,lecturer
Datetime,,,,,,,,,
2024-03-25 09:00:00,617.0,61.2,43.0,1023.1,1,1,0,15.0,0
2024-03-25 09:01:00,630.0,62.3,42.0,1023.1,1,1,0,15.0,0
2024-03-25 09:02:00,656.0,63.3,42.0,1023.0,1,1,0,18.0,0
2024-03-25 09:03:00,647.0,64.1,41.0,1023.1,1,1,0,18.0,0
2024-03-25 09:04:00,719.0,64.8,41.0,1023.1,1,1,0,18.0,0
...,...,...,...,...,...,...,...,...,...
2024-03-25 10:11:00,736.0,72.8,32.0,1022.9,0,1,1,19.0,0
2024-03-25 10:12:00,801.0,72.8,32.0,1023.0,0,1,1,19.0,0
2024-03-25 10:13:00,810.0,72.8,32.0,1023.0,0,1,1,19.0,0


In [186]:
windows['Chen'][5]

,co2,tempF,rhumid,atmpr,door1,door2,hvac,subject_count,lecturer
Datetime,,,,,,,,,
2024-04-10 09:00:00,766.0,71.7,43.0,1018.5,1,1,0,9.0,0
2024-04-10 09:01:00,727.0,71.5,44.0,1018.4,1,1,0,9.0,0
2024-04-10 09:02:00,741.0,71.3,44.0,1018.5,1,1,0,12.0,0
2024-04-10 09:03:00,764.0,71.2,44.0,1018.6,1,0,0,12.0,0
2024-04-10 09:04:00,802.0,71.1,45.0,1018.5,1,0,0,12.0,0
...,...,...,...,...,...,...,...,...,...
2024-04-10 10:11:00,829.0,72.2,42.0,1018.8,0,0,0,16.0,0
2024-04-10 10:12:00,865.0,72.2,42.0,1018.9,0,0,0,16.0,0
2024-04-10 10:13:00,852.0,72.2,42.0,1018.9,0,0,0,16.0,0


In [184]:
# Convert the dictionary of DataFrames to a dictionary of NumPy arrays, including the index
windows_np = {label: [(df.reset_index()).to_numpy() for df in windows_list] for label, windows_list in windows.items()}

# Get the column names from the first DataFrame, including the index column
column_names = ['Datetime'] + windows[next(iter(windows))][0].columns.tolist()

# Save the dictionary of NumPy arrays as a .npz file along with the column names
np.savez_compressed('../datasets/windows.npz', column_names=column_names, **windows_np)
